# 🧪 Laboratorio — Módulo 10: Reorganizar un mini proyecto con módulos ES

En este laboratorio vas a diseñar y crear **a mano** una pequeña estructura de ficheros TypeScript,
usando `import` / `export`, *barrel files* e intentando separar responsabilidades.

⚠️ Importante:
- No vamos a crear archivos desde el notebook.
- Este notebook es una **guía paso a paso**.
- Debes crear las carpetas y archivos en tu Codespace / entorno de trabajo.

---
## 🎯 Objetivo general

Construir un mini proyecto con esta estructura:

```text
src/
  models/
    Producto.ts
    Carrito.ts

  utils/
    precios.ts

  services/
    CheckoutService.ts

  index.ts
```

Y que `index.ts` pueda:
- Crear productos
- Crear un carrito
- Calcular el total
- Simular un checkout

---

# 🧱 Paso 1 — Crear la estructura de carpetas

🎯 **Objetivo de aprendizaje**: entender la separación por capas: `models`, `utils`, `services`.

En tu proyecto (por ejemplo en la raíz del repo), crea esta estructura:

```bash
mkdir -p src/models
mkdir -p src/utils
mkdir -p src/services
```

Tu árbol debería verse así:

```text
src/
  models/
  utils/
  services/
```

A continuación, iremos rellenando cada archivo con código.

# 🧱 Paso 2 — Definir el modelo `Producto`

📁 Archivo: `src/models/Producto.ts`

🎯 **Objetivo de aprendizaje**: crear un módulo que exporta un tipo y una clase relacionados.

1. Crea el archivo `Producto.ts` en la carpeta `models`.
2. Copia el siguiente código en `Producto.ts`:

```ts
export interface ProductoProps {
  id: string;
  nombre: string;
  precio: number;
}

export class Producto {
  id: string;
  nombre: string;
  precio: number;

  constructor(props: ProductoProps) {
    this.id = props.id;
    this.nombre = props.nombre;
    this.precio = props.precio;
  }
}
```

✔ Este módulo expone tanto el **tipo de construcción** (`ProductoProps`) como la **clase de dominio** (`Producto`).

# 🧱 Paso 3 — Definir el modelo `Carrito`

📁 Archivo: `src/models/Carrito.ts`

🎯 **Objetivo de aprendizaje**: ver cómo un modelo importa a otro y encapsula lógica.

1. Crea el archivo `Carrito.ts` en `src/models`.
2. Copia este código:

```ts
import { Producto } from "./Producto.ts";

export interface LineaCarrito {
  producto: Producto;
  cantidad: number;
}

export class Carrito {
  private lineas: LineaCarrito[] = [];

  agregar(producto: Producto, cantidad: number = 1) {
    const existente = this.lineas.find(l => l.producto.id === producto.id);
    if (existente) {
      existente.cantidad += cantidad;
    } else {
      this.lineas.push({ producto, cantidad });
    }
  }

  obtenerLineas() {
    return [...this.lineas];
  }
}
```

✔ Aquí ya estás usando `import { Producto }` desde otro módulo del mismo directorio (`models`).

# 🧱 Paso 4 — Utilidades de precios (`utils/precios.ts`)

📁 Archivo: `src/utils/precios.ts`

🎯 **Objetivo de aprendizaje**: separar lógica reutilizable de cálculo en un módulo de utilidades.

1. Crea el archivo `precios.ts` en `src/utils`.
2. Copia este código:

```ts
import type { LineaCarrito } from "../models/Carrito.ts";

export function calcularSubtotal(linea: LineaCarrito): number {
  return linea.producto.precio * linea.cantidad;
}

export function calcularTotal(lineas: LineaCarrito[]): number {
  return lineas.reduce((suma, linea) => suma + calcularSubtotal(linea), 0);
}

export function aplicarDescuento(total: number, porcentaje: number): number {
  return total - (total * porcentaje) / 100;
}
```

✔ Usamos `import type` para dejar claro que solo necesitamos los **tipos** de `Carrito` aquí.

# 🧱 Paso 5 — Servicio de checkout (`services/CheckoutService.ts`)

📁 Archivo: `src/services/CheckoutService.ts`

🎯 **Objetivo de aprendizaje**: ver un servicio que orquesta modelos y utilidades.

1. Crea el archivo `CheckoutService.ts` en `src/services`.
2. Copia el siguiente código:

```ts
import { Carrito } from "../models/Carrito.ts";
import { calcularTotal, aplicarDescuento } from "../utils/precios.ts";

export interface ResultadoCheckout {
  totalSinDescuento: number;
  totalConDescuento: number;
  descuentoAplicado: number;
}

export class CheckoutService {
  constructor(private carrito: Carrito) {}

  checkout(porcentajeDescuento: number = 0): ResultadoCheckout {
    const lineas = this.carrito.obtenerLineas();
    const totalSinDescuento = calcularTotal(lineas);
    const totalConDescuento = aplicarDescuento(totalSinDescuento, porcentajeDescuento);

    return {
      totalSinDescuento,
      totalConDescuento,
      descuentoAplicado: porcentajeDescuento,
    };
  }
}
```

✔ Este módulo **no conoce detalles de Producto**, solo trabaja con `Carrito` y `precios`.
Así se consigue una estructura orientada a responsabilidades.

# 🧱 Paso 6 — Punto de entrada `index.ts`

📁 Archivo: `src/index.ts`

🎯 **Objetivo de aprendizaje**: crear un **punto de entrada** que use todos los módulos.

1. Crea el archivo `index.ts` en `src`.
2. Copia el siguiente código:

```ts
import { Producto } from "./models/Producto.ts";
import { Carrito } from "./models/Carrito.ts";
import { CheckoutService } from "./services/CheckoutService.ts";

const p1 = new Producto({ id: "p1", nombre: "Teclado", precio: 30 });
const p2 = new Producto({ id: "p2", nombre: "Ratón", precio: 20 });

const carrito = new Carrito();
carrito.agregar(p1, 2);
carrito.agregar(p2, 1);

const checkoutService = new CheckoutService(carrito);
const resultado = checkoutService.checkout(10);

console.log("Carrito procesado:", resultado);
```

✔ Este archivo representa el **uso real** de tu mini dominio de negocio.

# 🧱 Paso 7 — (Opcional) Barrel files

📁 Archivo: `src/models/index.ts`

🎯 **Objetivo de aprendizaje**: entender cómo simplificar imports con un barrel.

1. Crea `src/models/index.ts` con:

```ts
export * from "./Producto.ts";
export * from "./Carrito.ts";
```

2. Modifica `src/index.ts` para importar desde el barrel:

```ts
import { Producto, Carrito } from "./models/index.ts";
import { CheckoutService } from "./services/CheckoutService.ts";

// resto igual...
```

✔ Esto muestra cómo un barrel puede simplificar los imports, pero a costa de introducir un archivo más.

# ✅ Comprobación manual

Según el entorno que estés usando, puedes:

### 🟢 Con Deno

Desde la raíz del proyecto:

```bash
deno run src/index.ts
```

Deberías ver algo similar a:

```text
Carrito procesado: { totalSinDescuento: 80, totalConDescuento: 72, descuentoAplicado: 10 }
```
### Usando el Notebook



In [ ]:
import { Producto } from "./src/models/Producto.ts";
import { Carrito } from "./src/models/Carrito.ts";
import { CheckoutService } from "./src/services/CheckoutService.ts";

const p1 = new Producto({ id: "p1", nombre: "Teclado", precio: 30 });
const p2 = new Producto({ id: "p2", nombre: "Ratón", precio: 20 });

const carrito = new Carrito();
carrito.agregar(p1, 2);
carrito.agregar(p2, 1);

const checkoutService = new CheckoutService(carrito);
const resultado = checkoutService.checkout(10);

console.log("Carrito procesado:", resultado);

---
### 🧩 Reflexión

- ¿Qué parte del código consideras **modelo**, **lógica de negocio** y **capa de aplicación**?
- ¿Dónde tendría sentido añadir validaciones adicionales?
- ¿Cómo podrías extraer otra utilidad a `utils/` si empiezas a repetir lógica?

---
🎉 **Laboratorio del Módulo 10 completado**

Has practicado:
- Diseñar una estructura modular de archivos TS
- Separar modelos, utilidades y servicios
- Usar `import` / `export` en un pequeño dominio de negocio
- Introducir barrel files para agrupar modelos

Esto deja el terreno preparado para el **módulo final de build y configuración**, donde usarás `tsconfig`, bundlers y scripts reales.